In [1]:
# !pip install langchain-huggingface chromadb pandas langchain sentence-transformers langchain-community langchain_openai langchain_chroma

In [2]:
import pandas as pd
import chromadb
from chromadb.utils import embedding_functions
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Chroma
from langchain.docstore.document import Document

In [4]:
# 1. 데이터 로드

ingredients_df = pd.read_csv("preprocessed_ingredients.csv")
cosmetics_df = pd.read_csv("oliveyoung_final.csv")

In [6]:
# 2. 임베딩 모델 준비
embedding_model = HuggingFaceEmbeddings(
    model_name="sentence-transformers/all-MiniLM-L6-v2"
)

<ipython-input-6-8985b7b4a1c4>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [7]:
# 3. CSV -> LangChain Document로 변환
documents = []
cosmetics = []

In [8]:
# 성분 Document
for _, row in ingredients_df.iterrows():
    content = row['description']
    metadata = {
        "type": "ingredient",
        "name": row['ingredient']
    }
    documents.append(Document(page_content=content, metadata=metadata))

In [9]:
# 화장품 Document
for _, row in cosmetics_df.iterrows():
    content = f"제품명: {row['product_name']}, 성분: {row['ingredient']}, 리뷰: {row['reviews']}, 사용법: {row['usage']}"
    metadata = {
        "type": "cosmetic",
        "product_name": row['product_name']
    }
    documents.append(Document(page_content=content, metadata=metadata))

In [10]:
# 4. Chroma 벡터 DB로 저장

vectordb = Chroma.from_documents(
    documents=documents,
    embedding=embedding_model,
    persist_directory="./chroma_db"
)

vectordb.persist()
print("ChromaDB에 저장 완료")

ChromaDB에 저장 완료


<ipython-input-10-5b5d51d83359>:9: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()


In [12]:
!pip freeze > chroma_env_requirements.txt

# ChromaDB 로드

In [13]:
# ChromaDB 로드
chromadb = Chroma(
		persist_directory="./chroma_db",
		embedding_function=embedding_model
)

<ipython-input-13-49c27cdb1281>:2: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-chroma package and should be used instead. To use it run `pip install -U :class:`~langchain-chroma` and import as `from :class:`~langchain_chroma import Chroma``.
  chromadb = Chroma(


In [14]:
# 프롬프트
template_with_examples = """
당신은 화장품 전문가입니다. 아래 예시처럼 질문에 답변하세요.

[규칙]
1. 반드시 type이 'cosmetic'인 문서에서 리뷰(reviews)와 성분(ingredient)을 참고하여, 질문 의도에 적합한 답변을 반환합니다.
2. 가장 먼저 질문 & 답변 예시들을 참고하여 질문에 대한 적절한 답을 반환합니다.
3. 화장품 추천 시에는 화장품 3가지를 추천하고, 꼭 실제 존재하는 화장품의 제품명과 브랜드명이 포함되어 있어야 합니다.
4. 답변 시 화장품의 효과, 궁합 포인트, 주요 성분, 추천 이유, 세정력, 주요 리뷰 등에서 적절한 답변을 사용자에게 반환합니다.
5. 리뷰에서 효과, 만족도, 특징이 잘 드러나는 제품을 우선적으로 추천합니다.

[성분 정보 추가 시]
- 추천 제품의 주요 성분을 확인한 후, 해당 성분이 어떤 효능을 가지는지 가독성이 높도록 설명에 추가합니다.
- 성분 설명은 type이 'ingredient'인 문서의 정보를 바탕으로 하되, '어디 문서에 따르면' 같은 표현은 사용하지 않습니다.

[궁합 포인트 추가 시]
- 두 가지의 화장품의 성분을 파악하여 성분들이 낼 수 있는 시너지를 바탕으로 설명합니다.

[추천 이유 추가 시]
- 리뷰를 바탕으로 리뷰에서 해당 화장품이 좋은 이유를 찾고, 그 이유를 바탕으로 설명합니다.
- 무조건 실제 리뷰를 요약하여 설명합니다.

[세정력 추가 시]
- 5점 만점으로 표현합니다.
- 5점이라면 ★★★★★, 4점이라면 ★★★★, 3점이라면 ★★★, 2점이라면 ★★, 1점이라면 ★를 반환합니다.

[주요 리뷰 추가 시]
- 무조건 실제 리뷰를 요약하여 설명합니다.

[3. 답변 스타일]
- 아래 예시처럼 자세하고, 사용자 친화적인 문장으로 작성합니다.
- 모든 답변은 실제 문서에 기반해야 하며, 존재하지 않는 제품이나 성분 정보를 만들어내지 않습니다.
- 가독성이 좋게 답변을 반환합니다.

[질문 & 답변 예시1]
질문 : 내가 넘버즈인 글루타치온 흔적 앰플을 가지고 있는데 이거랑 비타민이 함량된 세럼을 같이 써도 돼?
답변:
넘버즈인 글루타치온 흔적 앰플과 비타민이 함량된 세럼을 함께 사용하는 것은 일반적으로 문제가 없지만, 두 제품의 주요 성분이 서로 충돌할 가능성이 있으므로 주의가 필요합니다.
특히 비타민 C와 같은 성분은 다른 활성 성분과 함께 사용 시 자극을 유발할 수 있으니, 사용 전 패치 테스트를 권장합니다.
두 제품을 동시에 사용할 때는 한 제품을 아침에, 다른 제품을 저녁에 나누어 사용하는 것도 좋은 방법입니다.

넘버즈인 글루타치온의 흔적 앰플과 궁합이 좋은 세럼은 다음과 같습니다.

1. 토리든 다이브인 저분자 히알루론산 세럼
   - 효과: 수분 공급 + 피부결 정돈
   - 궁합 포인트: 글루타치온 앰플 사용 전 수분을 채워주면 미백 흡수력과 지속력이 높아짐

2. 작터지 레드 블레미쉬 클리어 수딩 세럼
   - 효과: 진정 + 피부장벽 강화
   - 궁합 포인트: 민감하거나 여드름 흔적도 함께 관리하고 싶을 때 궁합이 좋음

3. 더랩바이블랑두 판테놀 세럼
   - 효과: 진정 + 수분 +  장벽 보호
   - 궁합 포인트: 글루타치온 계열 앰플과 함께 써도 자극없이 시너지 가능


[질문 & 답변 예시2]
질문: 잡티 제거에 좋은 성분을 가진 화장품은?
답변:
잡티 제거에 효과적인 성분으로는 비타민 C, 나이아신아마이드, 알부틴, 코직산 등이 있습니다.
이 성분들은 멜라닌 생성을 억제하고 피부 톤을 밝게 하는 데 도움을 줄 수 있습니다. 제품을 선택할 때 이러한 성분이 포함되어 있는지 확인해보세요.
다만, 피부 타입에 따라 반응이 다를 수 있으므로 사용 전 패치 테스트를 권장합니다.

잡티 제거에 좋은 성분을 가진 화장품은 다음과 같습니다.

1. 아이소이 잡티세럼
   - 주요 성분: 알부틴(기미, 주근깨, 잡티 완화, 피부 톤 균일하게 개선), 나이아신아마이드(색소 침착 예방 및 완화, 모공 축소, 피지 조절, 진정, 장벽 강화까지 다기능)
   - 추천 이유: 민감성 피부용 미백 세럼으로 국내외에서 입소문이 많은 제품

2. 넘버즈인 5번 글루타치온 흔적 앰플
   - 주요 성분: 글루타치온(톤 개선, 기미·주근깨 예방, 피부 노화 방지 및 해독 작용(디톡스)), 나이아신아마이드(색소 침착 예방 및 완화, 모공 축소, 피지 조절, 진정, 장벽 강화까지 다기능), 트라넥사믹애씨드(기미, 색소침착, 염증성 홍조 개선)
   - 추천 이유: 칙칙함·피부톤 불균형·잡티 개선용으로 인기

3. 구달 청귤 비타C 잡티케어 세럼
   - 주요 성분: 청귤 추출물(잡티·기미·칙칙함 개선, 탄력 & 광채 부여), 비타민C 유도체(피부 톤 개선, 미백, 탄력 증진, 항산화 효과)
   - 추천 이유: 피부를 맑고 화사하게, 잡티·홍조 케어에 적합


[질문 & 답변 예시3]
질문: 세척력 좋은 클렌징폼 추천해줘
답변:
세척력이 좋은 클렌징폼을 찾고 계시다면, 모공 속 노폐물까지 깔끔히 제거해주는 제품 위주로 추천드릴게요.

1. 아니스프리 화산송이 모공 클렌징 폼
   - 추천 이유: 제주 화산송이 파우더가 피지와 노폐물을 강력하게 흡착하여 모공까지 깨끗하게 세정해줌.
   - 세정력: ★★★★★

2. 라로슈포제 에빠끌라 퓨리파잉 클렌징 젤
   - 추천 이유: 징크 PCA(피지 조절 성분)와 라로슈포제 온천수가 과도한 유분을 잡아주며 자극 없이 세정함
   - 세정력: ★★★★

3. AHC 클렌징 폼 (퓨어 리얼 아이 크림 폼)
   - 추천 이유: 마데카소사이드와 히알루론산이 세정 후에도 피부를 진정시키고 보습을 유지시켜 당김 없이 클렌징 가능
   - 세정력: ★★★★


[질문 & 답변 예시4]
질문: 향없는 토너 있을까?
답변:
향이 없는(무향/무향료) 토너는 민감성 피부나 향에 민감한 분들에게 좋습니다.
아래에 무향 또는 향료 미첨가 제품 중에서 평이 좋은 토너들을 추천드릴게요

1. 라운드랩 1025 독도 토너
   - 추천 이유: 가벼운 워터 제형으로 빠르게 흡수되며, 피부 진정과 수분 공급에 효과적
   - 주요 리뷰
     - 자극 없이 부드럽게 각질을 제거해주고, 피부를 촉촉하게 유지시켜줘서 이제는 화장이 더 잘 먹게 됐습니다.
     - 민감성인 나에게도 전혀 자극적이지 않고 순하게 느껴졌으며, 워터 타입이라 가볍고 산뜻하게 사용할 수 있다.

2. 하루하루원더 블랙라이스 히알루로닉 무향 토너
   - 추천 이유: 무향료로 향에 민감한 분들도 사용 가능하며, 보습과 피부 탄력 개선에 도움을 줌
   - 주요 리뷰
    - 향료 알레르기 있는데 이 제품은 알레르기 없이 잘 바르고 있어요. 촉촉함이 좋습니다.
    - 물같이 무르지 않은 제형이라 그런지 수분감이 더 묵직함 느낌입니다. 각질제거 성분이 따로 안들어 있어서 여러번 레이어링하기에도 자극 없이 좋았어요."

3. 더마토리 하이포알러제닉 시카 토너
   - 추천 이유: 무향료, 무알코올, 저자극 테스트 완료로, 예민한 피부의 진정과 장벽 강화에 도움을 줌
   - 주요 리뷰
     - 무난하게 사용하기 좋은 토너라는 생각이 들어요. 향도 없고 물같은 제형이라 피부결 정돈해주는 데 딱 좋아요.


[질문 & 답변 예시5]
질문: 건성 피부가 겨울철에 사용할만한 크림 추천해줘
답변:
건성 피부는 겨울철에 특히 건조하고 민감해지기 쉬우므로, 보습력이 뛰어나고 피부 장벽을 강화해주는 크림을 사용하는 것이 중요합니다.

건성 피부에게 겨울철 사용할 크림으로 추천할 목록은 다음과 같습니다.

1. 니베아 크림 (파란통)
   - 주요 성분: 판테놀(피부 진정 및 보습 강화), 미네랄 오일(피부 표면에 보호막 형성하여 수분 증발 방지)
   - 추천 이유: 풍부한 보습력으로 건조한 겨울철 피부를 촉촉하게 유지, 합리적인 가격으로 가성비 우수
   - 주요 리뷰
     - 판테놀과 미네랄 오일이 피부 깊숙이 수분을 공급해주고, 피부 표면에 보호막을 형성해줘요.

2. 예쁜얼굴 수분크림
   - 주요 성분: 고농축 히알루론산(피부 깊숙한 보습 제공), 쉐어버터(피부 영양 공급 및 보습 유지), 알란토인(피부 진정 및 보호), EGF(피부 탄력 증진)
   - 추천 이유: 즉각적인 수분 충전과 피부결 정돈에 효과적, 민감성 피부도 자극 없이 사용 가능
   - 주요 리뷰
     - 즉각적인 수분충전, 민감성 피부도 자극 없이 사용, 메이크업 시 들뜸 없음, 산뜻한 마무리감

3. 더마토리 하이포알러제닉 시카 젤 크림
   - 주요 성분: 병풀 추출물(피부 진정 및 보호), 판테놀(피부 보습 및 장벽 강화)
   - 추천 이유: 무향료, 무알코올, 저자극 포뮬러로 민감한 피부에 적합, 가벼운 젤 타입으로 빠른 흡수와 산뜻한 마무리감 제공
   - 주요 리뷰
    - 무향료, 무알코올, 저자극 테스트 완료로, 예민한 피부의 진정과 장벽 강화에 도움을 줍니다.

---
[참고 문서]
{context}

[실제 질문]
{question}
"""

In [15]:
from langchain_core.prompts import ChatPromptTemplate
from langchain.retrievers import EnsembleRetriever

prompt = ChatPromptTemplate.from_messages([
    ("system", template_with_examples),
    ("human", "{question}")
])

In [121]:
# retriver 만들기
retriever1 = chromadb.as_retriever(search_kwargs={"k": 2})
retriever2 = cosmetic_chromadb.as_retriever(search_kwargs={"k": 2}) # token 초과로 인해 사용 X

In [122]:
# 두 retriever를 하나로 묶기
# token 초과로 인해 사용 X
ensemble_retriever = EnsembleRetriever(
    retrievers=[retriever1, retriever2],
    weights=[0.5, 0.5]
)

In [48]:
from google.colab import userdata
import os
HF_TOKEN = userdata.get('HF_TOKEN')
openai_api_key = userdata.get('OPENAI_API_KEY')

In [104]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, openai_api_key=openai_api_key)

In [137]:
from langchain.chains import RetrievalQA

qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=retriever1,
    chain_type="stuff",
    chain_type_kwargs={"prompt": prompt},
    return_source_documents=True
)

In [159]:
qa_100_df = pd.read_csv("qa_100.csv")

In [160]:
qa_100_df['source']= ''

In [161]:
qa_100_df.head()

,question,answer,source
0,라운드랩 마스크팩 가지고 있는데 이거랑 궁합 잘 맞는 기초 화장품 있을까?,NaN,
1,복합성 피부가 진정 효과 있는 마스크팩 추천해줘,NaN,
2,건성 피부가 자극 없는 마스크팩 추천해줘,NaN,
3,토리든 앰플은 민감성 피부가 써도 괜찮아?,NaN,
4,피부가 얇은 편인데 순한 크림 있을까?,NaN,


In [189]:
for i, query in enumerate(qa_100_df['question']):
    try:
        result = qa_chain({"query": query})
        qa_100_df.at[i, 'answer'] = result["result"]
        qa_100_df.at[i, 'source'] = result["source_documents"]
        if i % 10 == 0:
          print(f'{i}개 완료')
    except Exception as e:
        print(f"Error on index {i}: {e}")
    time.sleep(2)

0개 완료
10개 완료
20개 완료
30개 완료
40개 완료
50개 완료
60개 완료
70개 완료
80개 완료
90개 완료


In [190]:
qa_100_df[84:]

,question,answer,source
84,건성이 겨울에 바르기 좋은 토너 추천해줘,"건성 피부는 겨울철에 특히 건조해지기 쉬우므로, 수분 공급과 보습력이 뛰어난 토너를...",[page_content='리드아세테이트트라이하이드레이트는 주로 염색약에서 착색제로...
85,내가 넘버즈인 글루타치온 흔적 앰플을 가지고 있는데 이거랑 비타민이 함량된 세럼을 ...,넘버즈인 글루타치온 흔적 앰플과 비타민이 함량된 세럼을 함께 사용하는 것은 일반적으...,[page_content='세틸베타인은 주로 헤어 컨디셔너와 클렌저에 사용되는 양이...
86,토리든 앰플이랑 궁합 안 맞는 제품 있을까?,토리든 앰플과 궁합이 맞지 않을 수 있는 제품을 선택할 때는 주의가 필요합니다. 토...,"[page_content='라우로카프람은 피부 투과 촉진제로 사용되며, 주로 화장품..."
87,민감성 피부인데 세라마이드 들어간 제품 괜찮을까?,세라마이드는 민감성 피부에 매우 적합한 성분입니다. 세라마이드는 피부 장벽을 강화하...,[page_content='비터오렌지꽃오일은 주로 항염 및 살균 효과로 피부 진정과...
88,여름에 트러블 많은데 괜찮은 선크림 뭐 있어?,"여름철 트러블이 많은 피부에는 자극이 적고, 유분을 조절해주는 선크림을 사용하는 것...",[page_content='하이드롤라이즈드돼지피용해추출물은 콜라겐 및 아미노산이 풍...
89,수부지 피부가 써도 괜찮은 촉촉한 세럼 추천해줘,"수부지 피부는 수분 부족형 지성 피부로, 수분 공급과 유분 조절이 동시에 필요합니다...",[page_content='팥꽃나무싹추출물은 피부 진정 및 항염 효과로 피부 관리 ...
90,피부결 개선에 좋은 토너 추천해줄 수 있어?,"피부결 개선에 효과적인 토너를 찾고 계시다면, 각질 제거와 수분 공급을 동시에 해주...",[page_content='리드아세테이트트라이하이드레이트는 주로 염색약에서 착색제로...
91,안자극 없이 쓸 수 있는 클렌징폼 뭐가 있어?,"안자극 없이 사용할 수 있는 클렌징폼을 찾고 계시다면, 민감한 눈가에도 자극 없이 ...",[page_content='납설포크로메이트옐로우는 안료로 사용되는 노란색 염료입니다...
92,복합성 피부인데 각질 안 일어나게 해줄 수 있는 제품 있어?,"복합성 피부는 T존은 기름지고 U존은 건조한 특징이 있어, 각질 관리가 중요합니다....",[page_content='중국으아리뿌리추출물은 항염 및 항균 특성이 있어 피부 진...
93,나이아신아마이드랑 잘 맞는 세럼 있으면 알려줘,"나이아신아마이드는 색소 침착 예방 및 완화, 모공 축소, 피지 조절, 진정, 장벽 ...",[page_content='리드아세테이트트라이하이드레이트는 주로 염색약에서 착색제로...


In [192]:
qa_100_df.to_csv("qa_100_df.csv", index=False)

In [193]:
len(qa_100_df)

100